In [1]:
import os
import glob
import itertools
import pickle

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [2]:
# Read in daily data
data_path = os.path.join('.', 'data')
daily_data_path = os.path.join(data_path, 'day_approach_maskedID_timeseries.csv')
weekly_data_path = os.path.join(data_path, 'week_approach_maskedID_timeseries.csv')

daily_df = pd.read_csv(daily_data_path)
daily_df.head()

,nr. sessions,total km,km Z3-4,km Z5-T1-T2,km sprinting,strength training,hours alternative,perceived exertion,perceived trainingSuccess,perceived recovery,nr. sessions.1,total km.1,km Z3-4.1,km Z5-T1-T2.1,km sprinting.1,strength training.1,hours alternative.1,perceived exertion.1,perceived trainingSuccess.1,perceived recovery.1,nr. sessions.2,total km.2,km Z3-4.2,km Z5-T1-T2.2,km sprinting.2,strength training.2,hours alternative.2,perceived exertion.2,perceived trainingSuccess.2,perceived recovery.2,nr. sessions.3,total km.3,km Z3-4.3,km Z5-T1-T2.3,km sprinting.3,strength training.3,hours alternative.3,perceived exertion.3,perceived trainingSuccess.3,perceived recovery.3,nr. sessions.4,total km.4,km Z3-4.4,km Z5-T1-T2.4,km sprinting.4,strength training.4,hours alternative.4,perceived exertion.4,perceived trainingSuccess.4,perceived recovery.4,nr. sessions.5,total km.5,km Z3-4.5,km Z5-T1-T2.5,km sprinting.5,strength training.5,hours alternative.5,perceived exertion.5,perceived trainingSuccess.5,perceived recovery.5,nr. sessions.6,total km.6,km Z3-4.6,km Z5-T1-T2.6,km sprinting.6,strength training.6,hours alternative.6,perceived exertion.6,perceived trainingSuccess.6,perceived recovery.6,Athlete ID,injury,Date
0,1.0,5.8,0.0,0.6,1.2,0.0,0.00,0.11,0.00,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.0,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.00,0.15,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0,0,1
2,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.0,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,0,0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.10,0.00,0.17,0,0,3
4,1.0,0.0,0.0,0.0,0.0,0.0,1.08,0.08,0.00,0.18,1.0,16.4,10.0,0.0,0.0,1.0,0.00,0.11,0.00,0.17,1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.10,0.00,0.15,1.0,5.2,0.0,0.5,1.2,0.0,0.00,0.10,0.00,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.00,-0.01,-0.01,-0.01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.10,0.00,0.17,1.0,17.6,7.2,0.0,0.0,0.0,0.0,0.11,0.00,0.17,0,0,4


In [21]:
# Create model trainer class
from utils import Trainer

test_split=0.2
cv=5
scoring='f1'

ignore_cols = ['Athlete ID']
model_trainer = Trainer(daily_df, 'injury', ignore_cols=ignore_cols, test_split=test_split)

## Logistic Regression

In [22]:
model_trainer.add_model('standard', 'log_reg')
print(model_trainer.models[('log_reg', 'standard')])

model_trainer.cross_validate('standard', 'log_reg', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('log_reg', 'standard')]['test_score'].mean())

Model type:  log_reg
Scaling:  standard
Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])
Model type:  log_reg
Scaling:  standard
Average test recall score:  0.0


## Complement Naive-Bayes

In [8]:
model_trainer.add_model('min_max', 'comp_nb')
print(model_trainer.models[('comp_nb', 'min_max')])

model_trainer.cross_validate('min_max', 'comp_nb', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('comp_nb', 'min_max')]['test_score'].mean())

Model type:  comp_nb
Scaling:  min_max
Pipeline(steps=[('scaler', MinMaxScaler()), ('model', ComplementNB())])
Model type:  comp_nb
Scaling:  min_max
Average test recall score:  0.03689406975124774


## K-Nearest Neighbors

In [12]:
weights='distance'
n_neighbors=10
model_trainer.add_model('min_max', 'knn', n_neighbors=n_neighbors, weights=weights)
print(model_trainer.models[('knn', 'min_max')])

model_trainer.cross_validate('min_max', 'knn', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('knn', 'min_max')]['test_score'].mean())

Model type:  knn
Scaling:  min_max
Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])
Model type:  knn
Scaling:  min_max
Average test recall score:  0.0


## Support Vector Machine

In [19]:
C=5.0
class_weight='balanced'
kernel='rbf'
model_trainer.add_model('min_max', 'svm', C=C, class_weight=class_weight, kernel=kernel)
print(model_trainer.models[('svm', 'min_max')])

model_trainer.cross_validate('min_max', 'svm', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('svm', 'min_max')]['test_score'].mean())

Model type:  svm
Scaling:  min_max
Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model', SVC(C=5.0, class_weight='balanced'))])
Model type:  svm
Scaling:  min_max
Average test recall score:  0.03774138389402519


In [20]:
model_trainer.scores[('svm', 'min_max')]['test_score']

array([0.05076142, 0.01891253, 0.02659574, 0.04494382, 0.0474934 ])

## Random Forest Classifier

In [10]:
class_weight='balanced'
criterion='entropy'
bootstrap=False
model_trainer.add_model(None, 'rf', class_weight=class_weight, bootstrap=bootstrap, criterion=criterion)
print(model_trainer.models[('rf', None)])

model_trainer.cross_validate(None, 'rf', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('rf', None)]['test_score'].mean())

Model type:  rf
Scaling:  None
Pipeline(steps=[('model',
                 RandomForestClassifier(bootstrap=False,
                                        class_weight='balanced',
                                        criterion='entropy'))])
Model type:  rf
Scaling:  None
Average test recall score:  0.0


## XGBoost Classifier

In [6]:
use_label_encoder=False
model_trainer.add_model(None, 'xgb', use_label_encoder=False)
print(model_trainer.models[('xgb', None)])

def f1_xgb(y_pred, d_train):
    y_true = d_train.get_label()
    return 'f1', f1_score(y_true, y_pred)

fit_params = {'eval_metric' : f1_xgb}
model_trainer.cross_validate(None, 'xgb', fit_params=fit_params, cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print('Average test recall score: ', model_trainer.scores[('xgb', None)]['test_score'].mean())

Model type:  xgb
Scaling:  None
Pipeline(steps=[('model',
                 XGBClassifier(base_score=None, booster=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, gamma=None, gpu_id=None,
                               importance_type='gain',
                               interaction_constraints=None, learning_rate=None,
                               max_delta_step=None, max_depth=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, reg_alpha=None,
                               reg_lambda=None, scale_pos_weight=None,
                               subsample=None, tree_method=None,
                               use_label_encoder=False,
                               validate_parameters

# Imbalanced data

## 1. Undersampling methods

### a. Cluster centroids

In [38]:
# Create model trainer class for imbalanced data
from utils import ImbTrainer

test_split=0.2
cv=5
scoring='f1'

ignore_cols = ['Athlete ID']
model_trainer = ImbTrainer(daily_df, 'injury', ignore_cols=ignore_cols, test_split=test_split)

### Logistic Regression

In [20]:
model_trainer.add_model('standard', 'clust_cents', 'log_reg')
print(model_trainer.models[('log_reg', 'standard', 'clust_cents')])

model_trainer.cross_validate('standard', 'clust_cents', 'log_reg', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print(f'Average test {scoring.upper()} score: ', model_trainer.scores[('log_reg', 'standard', 'clust_cents')]['test_score'].mean())

Model type:  log_reg
Scaling:  standard
Sampler type:  clust_cents
Pipeline(steps=[('scaler', StandardScaler()), ('sampler', ClusterCentroids()),
                ('model', LogisticRegression())])
Model type:  log_reg
Scaling:  standard
Sampler type:  clust_cents
Average test recall score:  0.03595304566786586


### K-Nearest Neighbors

In [43]:
model_params = {'weights': 'distance',
                'n_neighbors': 5}
model_trainer.add_model('min_max', 'clust_cents', 'knn', model_params=model_params)
print(model_trainer.models[('knn', 'min_max', 'clust_cents')])

model_trainer.cross_validate('min_max', 'clust_cents', 'knn', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print(f'Average test {scoring.upper()} score: ', model_trainer.scores[('knn', 'min_max', 'clust_cents')]['test_score'].mean())

Model type:  knn
Scaling:  min_max
Sampler type:  clust_cents
Pipeline(steps=[('scaler', MinMaxScaler()), ('sampler', ClusterCentroids()),
                ('model', KNeighborsClassifier(weights='distance'))])
Model type:  knn
Scaling:  min_max
Sampler type:  clust_cents
Average test F1 score:  0.024458803701581423


### Support Vector Machine

In [48]:
# C=5.0
# class_weight='balanced'
# kernel='rbf'
model_trainer.add_model('min_max', 'clust_cents', 'svm')
print(model_trainer.models[('svm', 'min_max', 'clust_cents')])

model_trainer.cross_validate('min_max', 'clust_cents', 'svm', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print(f'Average test {scoring.upper()} score: ', model_trainer.scores[('svm', 'min_max', 'clust_cents')]['test_score'].mean())

Model type:  svm
Scaling:  min_max
Sampler type:  clust_cents
Pipeline(steps=[('scaler', MinMaxScaler()), ('sampler', ClusterCentroids()),
                ('model', SVC())])
Model type:  svm
Scaling:  min_max
Sampler type:  clust_cents
Average test recall score:  0.036295151443745635


### Random Forests

In [53]:
# class_weight='balanced'
# criterion='entropy'
# bootstrap=False
model_params = {'bootstrap': False}
model_trainer.add_model('min_max', 'clust_cents', 'rf', model_params)
print(model_trainer.models[('rf', 'min_max', 'clust_cents')])

model_trainer.cross_validate('min_max', 'clust_cents', 'rf', cv=cv, scoring=scoring, return_train_score=True, return_estimator=True)
print(f'Average test {scoring.upper()} score: ', model_trainer.scores[('rf', 'min_max', 'clust_cents')]['test_score'].mean())

Model type:  rf
Scaling:  min_max
Sampler type:  clust_cents
Pipeline(steps=[('scaler', MinMaxScaler()), ('sampler', ClusterCentroids()),
                ('model', RandomForestClassifier(bootstrap=False))])
Model type:  rf
Scaling:  min_max
Sampler type:  clust_cents
Average test recall score:  0.02693425384906928


## Test out cluster centroids

In [35]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids()
test_X_res, test_y_res = cc.fit_resample(model_trainer.X_train, model_trainer.y_train)

In [33]:
print('Number of negative samples: ', (model_trainer.y_train == 0).sum())
print('Number of positive samples: ', (model_trainer.y_train == 1).sum())

Number of negative samples:  33747
Number of positive samples:  467


In [36]:
print('Number of negative samples: ', (test_y_res == 0).sum())
print('Number of positive samples: ', (test_y_res == 1).sum())

Number of negative samples:  467
Number of positive samples:  467


## Feature selection

In [158]:
# TODO: low variance features, correlated features, univariate feature selection, pca -> https://towardsdatascience.com/feature-selection-and-dimensionality-reduction-f488d1a035de